In [3]:
# !pip3 install torch
# !pip3 install numpy

import torch
import random
import numpy as np
import torch as torch
from torch.utils.data import dataloader, dataset
import torchvision
#
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.models.detection import transform
from torchvision import transforms
torch.backends.cudnn.deterministic = True

In [5]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [7]:
import csv
import os
from re import search

def make_img_folder(dataPath, csvPath, imgFolder):
    with open(csvPath) as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0].isdigit():
                print(f"cp {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
                os.system(f"mv {dataPath}/{'.'.join([row[0],'png'])} {imgFolder}/{row[1]}/")
Data_path = "/home/alexsun/diskb/streamlabs/task/images"
csv_path = "/home/alexsun/diskb/streamlabs/task/images_labelling.csv"
imgFolder = "/home/alexsun/diskb/streamlabs/task/data"
# make_img_folder(Data_path, csv_path, imgFolder)

In [20]:
dataset = torchvision.datasets.ImageFolder(imgFolder,transform = transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()]))
batch_size = 32
validation_split = .3
shuffle_dataset = True
random_seed= 8

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset,
                                                sampler=valid_sampler)
dataset

Dataset ImageFolder
    Number of datapoints: 3750
    Root location: /home/alexsun/diskb/streamlabs/task/data
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               ToTensor()
           )

In [23]:
from torchvision import datasets, transforms as T
import torch

model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model.eval()

# first_conv_layer = [torch.nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
# first_conv_layer.extend(list(model.children()))
# model.features= torch.nn.Sequential(*first_conv_layer)
# output = model(x)
# modules = list(model.children())[:-1]      # delete the last fc layer.
# model = torch.nn.Sequential(*first_conv_layer)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

for epoch in range(10000):
    for i, btch in enumerate(train_loader):
        x_train, labels_train = btch
        # resize = torchvision.transforms.Resize(x_train[0].shape)
        # x_train = resize(x_train)
        x_train = torch.Tensor(x_train).float()
        optimizer.zero_grad()

        preds = model.forward(x_train)

        loss_value = loss(preds, labels_train)
        loss_value.backward()

        optimizer.step()

    # acc = []
    x_test, target = validation_loader
    test_preds = model.forward(x_test)
    # test_loss_history.append(loss(test_preds, target))
    acc=(test_preds.argmax(dim=1) == target).float()
    # train_preds = mnist_net.forward(X_train)
    # train_loss_history.append(loss(train_preds, y_train))

    # print(acc)
    accuracy =acc.mean()
    # accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    # accuracy = (train_preds.argmax(dim=1) == y_train).float().mean()
    # test_accuracy_history.append(accuracy)
    # train_accuracy_history.append(accuracy)
    print(accuracy)


Using cache found in /home/alexsun/.cache/torch/hub/pytorch_vision_v0.6.0


KeyboardInterrupt: 